<a href="https://colab.research.google.com/github/willystw/fastai-learning/blob/bear-multi-classification/movie-recommender/movie_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#hide
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

     |████████████████████████████████| 720 kB 4.2 MB/s 
     |████████████████████████████████| 46 kB 4.1 MB/s 
     |████████████████████████████████| 1.2 MB 38.3 MB/s 
     |████████████████████████████████| 186 kB 52.8 MB/s 
     |████████████████████████████████| 56 kB 3.6 MB/s 
     |████████████████████████████████| 51 kB 312 kB/s 
Mounted at /content/gdrive


In [ ]:
#hide
from fastbook import *

In [ ]:
from fastai.collab import *
from fastai.tabular.all import *
path = untar_data(URLs.ML_100k)

In [ ]:
ratings = pd.read_csv(path/'u.data', delimiter='\t', header=None, names=['user','movie','rating','timestamp'])
ratings.head(10)

,user,movie,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
5,298,474,4,884182806
6,115,265,2,881171488
7,253,465,5,891628467
8,305,451,3,886324817
9,6,86,3,883603013


In [ ]:
movies = pd.read_csv(path/'u.item', delimiter='|', header=None, usecols=(0,1), names=('movie', 'title'), encoding='latin-1')
movies.head(10)

,movie,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)
5,6,Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)
6,7,Twelve Monkeys (1995)
7,8,Babe (1995)
8,9,Dead Man Walking (1995)
9,10,Richard III (1995)


In [ ]:
ratings = ratings.merge(movies)
ratings.head(10)

,user,movie,rating,timestamp,title
0,196,242,3,881250949,Kolya (1996)
1,63,242,3,875747190,Kolya (1996)
2,226,242,5,883888671,Kolya (1996)
3,154,242,3,879138235,Kolya (1996)
4,306,242,5,876503793,Kolya (1996)
5,296,242,4,884196057,Kolya (1996)
6,34,242,5,888601628,Kolya (1996)
7,271,242,4,885844495,Kolya (1996)
8,201,242,4,884110598,Kolya (1996)
9,209,242,4,883589606,Kolya (1996)


In [ ]:
dls = CollabDataLoaders.from_df(ratings, item_name='title', bs=64)
dls.show_batch()

,user,title,rating
0,178,Days of Thunder (1990),3
1,92,2 Days in the Valley (1996),3
2,178,Michael Collins (1996),3
3,804,"Adventures of Pinocchio, The (1996)",3
4,738,Sleepless in Seattle (1993),3
5,583,Brazil (1985),5
6,878,Leaving Las Vegas (1995),3
7,742,"Postino, Il (1994)",5
8,694,When a Man Loves a Woman (1994),3
9,825,Mother (1996),3


In [ ]:
learn = collab_learner(dls, n_factors=50, y_range=(0, 5.5))

In [ ]:
learn.fit_one_cycle(5, 5e-3, wd=0.1)

epoch,train_loss,valid_loss,time
0,0.913259,0.943922,00:11
1,0.817469,0.873518,00:11
2,0.733714,0.829292,00:11
3,0.581267,0.816393,00:11
4,0.479274,0.817592,00:11


In [ ]:
learn = collab_learner(dls, use_nn=True, y_range=(0, 5.5), layers=[100,50])
learn.fit_one_cycle(5, 5e-3, wd=0.1)

epoch,train_loss,valid_loss,time
0,0.995056,0.985757,00:15
1,0.959629,0.906348,00:16
2,0.868694,0.877667,00:16
3,0.842771,0.852472,00:15
4,0.736606,0.854832,00:15
